# Bayesian inference of water gas shift using GP as prior

In [1]:
# load packages
import numpy as np 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, ConstantKernel as C
import utils

In [2]:
# GP train setting

# basis function
Basis = ['outer_electron', 'inner_shell', \
         'coord_num', 'valence', 'num_non_hydro', \
         'M-C', 'M-O', 'M-H', 'PW91RelEnergy', '#CH3OH', '#H2O', '#H2']
Nstart = 10000    # GP restart optimization

limit_C = 100;    # lower bound for noise level
limit_l = 1e-6    # lower bound for lengthscale
isotropic_kernel = False                           # ARD kernel

Scale = 'MinMax'  # standardize method
Functional = 'Err_RelEnergy'
WGSFxn = 'PW91BindEnergy'

gp_trainfile = './Benchmark_data/Campbell_Mavrikakis_rel_v1.csv'    # gp training file
metal_file = './Benchmark_data/metal_constant.csv'

In [3]:
# pre-calculated hyperparameters
hyp  = [7.767450383154461, 5.757596027338672, -0.1228293962244263, 13.009740697566935, -0.5650730981457128, 10.186156540562594, 11.525166763586267, 8.708465224255864, 4.749814718016643, -0.4903878616734594, 6.171236719199616, 10.708422581386317, -0.33423780342364817, 4.605170185988092]

In [4]:
# Load and preprocessing the data
sorp_data = utils.read_file(gp_trainfile)
utils.outer_electron(sorp_data, metal_file)
utils.inner_shell(sorp_data, metal_file)
utils.coordination_num(sorp_data, metal_file)

# build training data
Xtrain = [sorp_data[base] for base in Basis]
Xtrain = np.array(Xtrain).T 
Ndata, Nfeature = Xtrain.shape

# Scaling
if Scale == 'MinMax':
    mms = MinMaxScaler()
    Xtrain = mms.fit_transform(Xtrain)
elif Scale == 'Std':
    stdsc = StandardScaler()
    Xtrain = stdsc.fit_transform(Xtrain)
    
# Output
Ytrain = -np.array(sorp_data[Functional])

In [5]:
# Define the kernel and GP model 
if isotropic_kernel:
    k1 = C(1, (1e-6, 1e6)) * RBF(1, (limit_l, 1e6))
else:  
    k1 = C(1, (1e-6, 1e6)) * RBF(np.ones(Nfeature), (limit_l, 1e6))
k2 = WhiteKernel(noise_level=1,
                  noise_level_bounds=(100, 1e6))  # noise terms
kernel = k1+k2
gpr = GaussianProcessRegressor(kernel=kernel, optimizer=None)

hyp = np.array(hyp)
gpr.kernel.theta = hyp          # laod the hyperparameters
gpr.fit(Xtrain, Ytrain)

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
             kernel=48.6**2 * RBF(length_scale=[317, 0.884, 4.47e+05, 0.568, 2.65e+04, 1.01e+05, 6.05e+03, 116, 0.612, 479, 4.47e+04, 0.716]) + WhiteKernel(noise_level=100),
             n_restarts_optimizer=0, normalize_y=False, optimizer=None,
             random_state=None)

In [6]:
# apply GP for the species applied in microkinetic model

gp_predictfile = './Benchmark_data/InputMetSynGP.csv'

# build predict input
GPpred_data = utils.read_file(gp_predictfile)
utils.site_dis(GPpred_data, metal_file)
utils.outer_electron(GPpred_data, metal_file)
utils.inner_shell(GPpred_data, metal_file)
utils.coordination_num(GPpred_data, metal_file)
Adsorbate = GPpred_data['Adsorbate']

# WGS Input
Xpred = [GPpred_data[base] for base in Basis]
Xpred = np.array(Xpred).T
Npred, _ = Xpred.shape

if Scale == 'MinMax':
    Xpred = mms.transform(Xpred)
elif Scale == 'Std':
    Xpred = stdsc.transform(Xpred)

# GP prediction on gp_predictfile
GPpred, CovGPpred = gpr.predict(Xpred, return_cov=True)

# Microkinetic model bayesian inference

In [7]:
# load kinetic module
from AbCD.model import CSTR
from AbCD.io_data import In_data

In [8]:
# load speices and reaction list and reaction condition
sitelist, specieslist, reactionlist = In_data.load_mkm('./WGSdata/')
conditionlist = In_data.load_condition('./WGSdata/')

# which reaction and species are defined are varied in microkinetics
dEa_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
dBE_index = [4, 5, 6, 7, 8, 9, 10, 11]

# define CSTR reactor
reactor = CSTR(specieslist=specieslist,
              reactionlist=reactionlist,
              dEa_index=dEa_index,
              dBE_index=dBE_index)
reactor.initialize()            # initialize the reactor

In [9]:
# determine the reaction condition need be inferred
act = [0]
conditionlist = [condi for ii, condi in enumerate(conditionlist) if ii in act]

In [10]:
# define prior information

HH = reactor.stoimat[:, dBE_index]    # active stroichiometric
    
mean_BE = [];
weight = np.zeros(len(Adsorbate));
nonz = []
for ii, spe in enumerate(specieslist):
    try:
        idx = Adsorbate.index(str(spe))
        mean_BE.append(GPpred[idx])
        spe.dE = GPpred[idx]
        weight[idx] = 1
        nonz.append(idx)
    except:
        pass

non_in = [i for i in range(len(Adsorbate)) if i not in nonz]

cov_BE = np.copy(CovGPpred)
cov_BE = np.delete(cov_BE, non_in, 0)
cov_BE = np.delete(cov_BE, non_in, 1)

omega_list = []
for rxn in reactionlist:
    omega_list.append(rxn.dft_data['omega'])
    
mean_Ea = np.diag(omega_list).dot(HH).dot(mean_BE)
mean_prior = np.concatenate([mean_Ea, mean_BE])

cov_Ea = 100 * np.eye(len(dEa_index))             # 10 as std for deviation in Ea
BE2Ea = np.diag(omega_list).dot(HH)

In [11]:
# construct evidence and prior dictionary
evidence = {}
evidence['type'] = 'rel'              # the error for TOF measurement is assumed to be relative error
evidence['err'] = 0.2                 # the std is assumed to be 20 %

prior = {}
prior['type'] = 'GP'
prior['BEmean'] = mean_BE
prior['BEcov'] = cov_BE
prior['BE2Ea'] = BE2Ea
prior['Eacov'] = cov_Ea

prior['lbound'] = [-100.] * (len(dEa_index) + len(dBE_index))
prior['ubound'] = [100.] * (len(dEa_index) + len(dBE_index))

In [12]:
dP = np.zeros(len(dEa_index) + len(dBE_index))
if not reactor.CheckThermoConsis(dP):    # check thermodynamic consistency
    dP = reactor.CorrThermoConsis(dP)    # correct thermodynamic consistency
Edis, tor_dis, result_dis = reactor.bayesian_infer(10000, 1000, dP, 1*np.eye(len(dEa_index) + len(dBE_index)),
                                                   conditionlist, evidence, prior,
                                                   sample_method='augment',
                                                   step_write=1000)


{'tf': 1000000, 'reltol': 1e-12, 'abstol': 1e-14, 'max_num_steps': 100000.0, 'disable_internal_warnings': True}
   step       log(prior)     log(likelihood)  log(posterior)       accept%         infeasi%    
    0           -18.51           -23.98           -42.48            0.00             0.00      
   1000         -11.72            -5.53           -17.25            46.05            0.00      
   2000          -4.62            -0.50            -5.13            43.03            0.00      
   3000          -9.43            -0.78           -10.22            41.89            0.00      
   4000         -12.71            -0.49           -13.20            41.49            0.00      
   5000         -11.15            -0.04           -11.19            41.79            0.00      
   6000          -8.49            -1.49            -9.98            42.64            0.00      
   7000          -9.22            -0.30            -9.52            42.94            0.00      
   8000          -8.44  

In [ ]:
# save the infer distribution
Edis = np.array(Edis)